In [2]:
import dlib
import numpy as np
import cv2
from math import hypot
from my_package import *

In [11]:
cap = cv2.VideoCapture(0)
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
both_blinking = False

keyboard = np.zeros((1600,800,3), np.uint8)

keys_set_1 = {0:"A", 1: "Z", 2 : "E", 3: "R", 4 :"T",
                5:"Q", 6: "S", 7 : "D", 8: "F", 9 :"G",
                10:"W", 11: "X", 12 : "C", 13: "V", 14 :"B"}

def draw_letter(letter_index, text, letter_light):
    if letter_index ==0:
        x = 0
        y= 0
    elif letter_index ==1:
        x = 200
        y =  0
    elif letter_index ==2:
        x = 400
        y =  0
    elif letter_index ==3:
        x = 600
        y =  0
    elif letter_index ==4:
        x = 800
        y =  0
    elif letter_index ==5:
        x = 0
        y =  200    
    elif letter_index ==6:
        x = 200
        y = 200 
    elif letter_index ==7:
        x = 400
        y = 200
    elif letter_index ==8:
        x = 600
        y = 200
    elif letter_index ==9:
        x = 800
        y = 200
    elif letter_index ==10:
        x = 0
        y = 400
    elif letter_index ==11:
        x = 200
        y = 400
    elif letter_index ==12:
        x = 400
        y = 400
    elif letter_index ==13:
        x = 600
        y = 400
    elif letter_index ==14:
        x = 800
        y = 400

    width = 200
    heigth = 200
    thickness = 3
    if letter_light is True:
            cv2.rectangle(keyboard, (x + thickness,y + thickness), (x + width - thickness,y +heigth - thickness),(255,255,255), -1)
    else :
            cv2.rectangle(keyboard, (x + thickness,y + thickness), (x + width - thickness,y +heigth - thickness),(255,0,0), thickness)


    font_letter = cv2.FONT_HERSHEY_PLAIN
    #Text settings
    font_scale =10

    font_thick = 4
    text_size = cv2.getTextSize(text, font_letter, font_scale, font_thick)[0]
    width_text, heigth_text = text_size[0],text_size[1]

    text_x = int((width - width_text)/2) + x
    text_y = int((heigth + heigth_text)/2) + y
    cv2.putText(keyboard, text, (text_x,text_y),font_letter, font_scale, (255,0,0), font_thick)

# Keyboards Settings

font = cv2.FONT_HERSHEY_PLAIN

def midpoint(p1, p2):
    return int((p1.x + p2.x) / 2), int((p1.y + p2.y)/2)

def get_blinking_ratio(eye_points, facial_landmarks):
    left_point = (facial_landmarks.part(eye_points[0]).x, facial_landmarks.part(eye_points[0]).y)
    right_point = (facial_landmarks.part(eye_points[3]).x, facial_landmarks.part(eye_points[3]).y)
        
    center_top = midpoint(facial_landmarks.part(eye_points[1]), facial_landmarks.part(eye_points[2]))
    center_bottom = midpoint(facial_landmarks.part(eye_points[5]), facial_landmarks.part(eye_points[4]))
        
    #hor_line = cv2.line(frame, left_point,right_point, (0,0,255), 2)
    #ver_line = cv2.line(frame, center_bottom,center_top, (0,0,255), 2)
        
    hor_line_length = hypot(left_point[0] - right_point[0], left_point[1] - right_point[1])
    ver_line_length = hypot(center_top[0] - center_bottom[0], center_top[1] - center_bottom[1])

    ratio_eye = hor_line_length/ver_line_length
 
    return ratio_eye

def get_gaze_ratio(eye_points, facial_landmarks):
    eye_region = np.array([(facial_landmarks.part(eye_points[0]).x, facial_landmarks.part(eye_points[0]).y),
                                    (facial_landmarks.part(eye_points[1]).x, facial_landmarks.part(eye_points[1]).y),
                                    (facial_landmarks.part(eye_points[2]).x, facial_landmarks.part(eye_points[2]).y),
                                    (facial_landmarks.part(eye_points[3]).x, facial_landmarks.part(eye_points[3]).y),
                                    (facial_landmarks.part(eye_points[4]).x, facial_landmarks.part(eye_points[4]).y),
                                    (facial_landmarks.part(eye_points[5]).x, facial_landmarks.part(eye_points[5]).y)], np.int32)
        
    height, width, _ = frame.shape
    mask = np.zeros((height,width),np.uint8)
    cv2.polylines(mask, [eye_region], True, 255, 2 )
    cv2.fillPoly(mask, [eye_region], 255)
    eye = cv2.bitwise_and(gray,gray, mask = mask)
        
        #cv2.polylines(frame, [left_eye_region], True, (0,255,0), 2 )
    min_x =np.min(eye_region[:,0])
    max_x =np.max(eye_region[:,0])
        
    min_y =np.min(eye_region[:,1])
    max_y =np.max(eye_region[:,1])
        
    gray_eye = eye[min_y: max_y, min_x: max_x]
    _, threshold_eye = cv2.threshold(gray_eye,70, 255, cv2.THRESH_BINARY_INV)
    height, width = threshold_eye.shape

    left_side_threshold = threshold_eye[0:height, 0: int(width/2)]
    right_side_threshold = threshold_eye[0:height, int(width/2): width]
    left_side_white = cv2.countNonZero(left_side_threshold)
    right_side_white = cv2.countNonZero(right_side_threshold)

    gaze_ratio = left_side_white/right_side_white
    return gaze_ratio

#Counters
frames = 0
letter_index = 0

while True:
    _, frame = cap.read()
    frame =cv2.flip(frame,1)
    keyboard[:] = (0,0,0)
    new_frame = np.zeros((500,500,3), np.uint8)
    frames += 1

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = detector(gray)
    for face in faces :
        #Réaliser rectangle du visage
        #x, y = face.left(), face.top()
        #x1, y1 = face.right(), face.bottom()
        #cv2.rectangle(frame, (x,y), (x1, y1), (0,255,0), 2)
        landmarks = predictor(gray, face)
        left_eye_ratio = get_blinking_ratio([36,37,38,39,40,41], landmarks)
        
        right_eye_ratio = get_blinking_ratio([42,43,44,45,46,47], landmarks)
        blinking_both_ratio = (left_eye_ratio + right_eye_ratio) /2

        if  blinking_both_ratio > 8:
            both_blinking = True
            cv2.putText(frame, "BOTH EYES BLINKING", (50,150), font, 3, (34,25,66))

        if  left_eye_ratio > 5.75 and not both_blinking:
            left_eye_blinking = True
        #    cv2.putText(frame, "LEFT EYE BLINKING", (50,150), font, 3, (0,255,0))
            active_letter = keys_set_1[letter_index]
            print(active_letter)
        if  right_eye_ratio > 5.75 and not both_blinking:
            right_eye_blinking = True
        #    cv2.putText(frame, "RIGHT EYE BLINKING", (50,150), font, 3, (0,0,255))
        

        #Gaze Ratio
        left_eye_gaze_ratio = get_gaze_ratio([36,37,38,39,40,41],landmarks)
        right_eye_gaze_ratio = get_gaze_ratio([42,43,44,45,46,47],landmarks)
        gaze_ratio = (right_eye_gaze_ratio + left_eye_gaze_ratio)/2
        

        if gaze_ratio <=1:
            cv2.putText(frame,"RIGHT", (50,150), font, 2, (0,0,255), 3)
            new_frame[:] = (0,0,255)

        elif 1< gaze_ratio < 2 :
            cv2.putText(frame,"CENTER", (50,150), font, 2, (0,0,255), 3)
        else :
            cv2.putText(frame,"LEFT", (50,150), font, 2, (0,0,255), 3)
            new_frame[:] = (255,0,0)


        cv2.putText(frame,str(gaze_ratio), (50,100), font, 2, (0,0,255), 3)

    #Letters
    if frames == 10:
        letter_index += 1
        frames = 0

    for i in range(15):
        if i == letter_index:
            light = True
        else:
            light = False
        draw_letter(i, keys_set_1[i],light)


        pass
    cv2.imshow("Frame", frame)
    cv2.imshow("New Frame", new_frame)
    cv2.imshow("Virtual Keyboard", keyboard)
    
    key =cv2.waitKey(1)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()

R
R
R
T
T
T
S
D
D
D
D
D
D
D
D
G
G
G
W
W
W
W
W
W
W
X
C
C
C
V
V
B
B
B
B
B
B


KeyError: 15

: 